In [10]:
import glob
import os
import shutil
import cv2
import matplotlib.pyplot as plt
import mediapipe as mp
from pathlib import Path


from ultralytics import YOLO

In [11]:
# # Renames all files of a folder

# from pathlib import Path


# c = 0

# directory = "/home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data"
# model_path = "/home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/runs/detect/train15/weights/best.pt"
# model = YOLO(model_path)

# for folder in glob.glob(f"{directory}/*"):
#     # Open file
#     # print("Folder: ", folder)
#     if folder.split("/")[-1] == "22":
#         for images in os.listdir(folder):
#             # check if the image ends with png
#             image_path = folder + "/" + images
#             print("image_path : ", image_path)
#             results = model(image_path)
#             print(results)
#             for result in results:
#                 print("image:")
#                 print(result.orig_img)
#                 print("result.boxes")
#                 print(result.boxes.cpu().numpy())
#                 print("\n\n\n")
#                 print("Required results:")
#                 print("Bounding boxes of right hand")
#                 print(result.boxes[0].xyxy.cpu().numpy())
#                 print("Bounding boxes for left hand")
#                 print(result.boxes[1].xyxy.cpu().numpy())
#                 print("In total result.boxes")
#                 print(result.boxes)
#                 print("\n\nresult.boxes[0]")
#                 print(result.boxes[0])
#                 print("\n\nresult.boxes[1]")
#                 print(result.boxes[1])
#                 print("\n\nresult.boxes[2]")
#                 print(result.boxes[2])
#                 print("The class of first bounding box:")
#                 print(int(result.boxes[0].cls.cpu().numpy()[0]))
#             results[0].show()

#             # result[1] is not there
#             # print("\n\n\nresults[1]")
#             # print(results[1])
#             break
#         break
#     else:
#         continue


In [12]:
def get_short(x, n):
    x = str(x)
    al = x.split(".")
    c = al[0] + "." + al[1][:n]
    return float(c)

In [ ]:
directory = "/home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data"
model_path = "/home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/runs/detect/train15/weights/best.pt"



# Global variables
width = 640
height = 480
resize_dim = 300
c = 0
img_bb_pad = 35
cv2_frame_size = [width, height]
data = []
labels = []





model = YOLO(model_path)





for folder in glob.glob(f"{directory}/*"):

    print("Folder: ", folder)

    for images in os.listdir(folder):

        # Taking the image path

        image_path = folder + "/" + images
        print("Image")
        # print("image_path : ", image_path)
        



        
        # Process the image for hand detection

        det_results = model(image_path)
        



        # To get a padded detection coordinates( only one for left, and one for right)

        left_area = 0
        left_coord = [0, 0, 0, 0]
        right_area = 0
        right_coord = [0, 0, 0, 0]

        for box in det_results[0].boxes:

            # Getting the basics

            bb = box.xyxy.cpu().numpy()[0]
            bb_class = int(box.cls.cpu().numpy()[0])
            bb_class_str = "Right_Hand" if bb_class else "Left_Hand"
            bb_area = (bb[2] - bb[0])*(bb[3]-bb[1])
            # print("Bounding box classification: ", bb_class_str)
            # print("Bounding box coordinates: ", bb)
            # print("Bounding box area: ", bb_area, "\n")
            if bb_class_str == "Left_Hand":
                if bb_area > left_area:
                    left_area = bb_area

                    bb_pad = [bb[0]-img_bb_pad, bb[1]-img_bb_pad, bb[2]+img_bb_pad, bb[3]+img_bb_pad]
                    final_bb_pad = [max(bb_pad[0], bb[0]), max(bb_pad[1], bb[1]), min(bb_pad[2], bb[2]), min(bb_pad[3], bb[3])]

                    left_coord = final_bb_pad
            else:
                if bb_area > right_area:
                    right_area = bb_area

                    bb_pad = [bb[0]-img_bb_pad, bb[1]-img_bb_pad, bb[2]+img_bb_pad, bb[3]+img_bb_pad]
                    final_bb_pad = [max(bb_pad[0], bb[0]), max(bb_pad[1], bb[1]), min(bb_pad[2], bb[2]), min(bb_pad[3], bb[3])]

                    right_coord = final_bb_pad

        # print("Final two bounding box:")
        # print("Left hand coordinates:")
        # print(left_coord)
        # print("\n")
        # print("Right hand coordinates:")
        # print(right_coord)







        #To crop, resize, and if needed to plot the padded detection boxes

        img = cv2.imread(image_path)
        # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        image_with_new_bounding = cv2.rectangle(
                                                img, 
                                                ( int(left_coord[0]), int(left_coord[1]) ),
                                                ( int(left_coord[2]), int(left_coord[3]) ),
                                                color=(0, 0, 255),
                                                thickness=2)
        image_with_new_bounding = cv2.rectangle(
                                                img, 
                                                ( int(right_coord[0]), int(right_coord[1]) ),
                                                ( int(right_coord[2]), int(right_coord[3]) ),
                                                color=(0, 255, 0),
                                                thickness=2)
        
        
        
        print("Found left and right bb")
        cropped_images = []
        cropped_images.append(img[ int(left_coord[1]) : int( left_coord[3] ), int( left_coord[0] ) : int( left_coord[2] )] )
        cropped_images.append(img[ int(right_coord[1]) : int( right_coord[3] ), int( right_coord[0] ) : int( right_coord[2] )] )


        l1 = len(cropped_images[0])
        r1 = len(cropped_images[1])






        # To get the landmarks of each hand for each image

        c += 1
        precision = 3
        temp_data = []
        left_data = []
        right_data = []
        fin_data = []
        fin_label = []


        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5)


        try:

            print("\n\n\nGoing to find keypoints: \n\n\n\n\n\n")
            for i in cropped_images:
                print("Taking element of cropped images")
                print("Len of the taken cropped image: ", len(i))
                print("Is the cropped image an empty list: ", "Yes" if i == [] else "No")
                try:
                    
                    file_path = "data1/left" + str(c) + ".png"
                    resized = cv2.resize(cropped_images[0], (resize_dim, resize_dim))
                    results = hands.process(resized)
                    left_hand_lm = results.multi_hand_landmarks[0]

                    for i in range(21):
                        # Left hand points
                        x1 = get_short(left_hand_lm.landmark[i].x, precision)
                        y1 = get_short(left_hand_lm.landmark[i].y, precision)
                        z1 = get_short(left_hand_lm.landmark[i].z, precision)

                        temp_data.append([x1, y1, z1])

                    for i in temp_data:
                        for j in range(3):
                            left_data.append(i[j])

                    cv2.imwrite(file_path, resized)

                except:
                    continue

                try:

                    file_path = "data1/right" + str(c) + ".png"
                    resized = cv2.resize(cropped_images[1], (resize_dim, resize_dim))

                    results = hands.process(resized)
                    right_hand_lm = results.multi_hand_landmarks[1]

                    for i in range(21):
                        # Left hand points
                        x1 = get_short(right_hand_lm.landmark[i].x, precision)
                        y1 = get_short(right_hand_lm.landmark[i].y, precision)
                        z1 = get_short(right_hand_lm.landmark[i].z, precision)

                        temp_data.append([x1, y1, z1])

                    for i in temp_data:
                        for j in range(3):
                            right_data.append(i[j])

                    # Right hand is detected
                    # print("right detected")

                    cv2.imwrite(file_path, resized)
                
                except:
                    continue

            if left_data != []:
                for i in left_data:
                    fin_data.append(i)
            else:
                for i in range(63):
                    fin_data.append(0)
            
            if right_data != []:
                for i in right_data:
                    fin_data.append(i)
            else:
                for i in range(63):
                    fin_data.append(0)

        except:
            continue


        fin_label.append(folder.split("/")[-1])


        print("Final data dimension: (Should be 126): ", len(fin_data))
        print("\n\n fin_data:\n", fin_data)
        print("Final label dimension: (should be 1): ", len(fin_label))
        print("\n\n fin_label:\n", fin_label)

        data.append(fin_data)
        labels.append(fin_label)

        # if l1 != 0:
        #     # Left hand is detected
        #     # print("left detected")
        #     file_path = "data1/left" + str(c) + ".png"
        #     resized = cv2.resize(cropped_images[0], (resize_dim, resize_dim))

        #     results = hands.process(resized)
        #     left_hand_lm = results.multi_hand_landmarks[0]

        #     for i in range(21):
        #         # Left hand points
        #         x1 = get_short(left_hand_lm.landmark[i].x, precision)
        #         y1 = get_short(left_hand_lm.landmark[i].y, precision)
        #         z1 = get_short(left_hand_lm.landmark[i].z, precision)

        #         temp_data.append([x1, y1, z1])

        #     for i in temp_data:
        #         for j in range(3):
        #             fin_data.append(i[j])

        #     cv2.imwrite(file_path, resized)
        # else:

        #     for i in range(63):
        #         fin_data.append(0)
        #     # print("left not detected")
        # if r1 != 0:

        #     file_path = "data1/right" + str(c) + ".png"
        #     resized = cv2.resize(cropped_images[1], (resize_dim, resize_dim))

        #     results = hands.process(resized)
        #     right_hand_lm = results.multi_hand_landmarks[1]

        #     for i in range(21):
        #         # Left hand points
        #         x1 = get_short(right_hand_lm.landmark[i].x, precision)
        #         y1 = get_short(right_hand_lm.landmark[i].y, precision)
        #         z1 = get_short(right_hand_lm.landmark[i].z, precision)

        #         temp_data.append([x1, y1, z1])

        #     for i in temp_data:
        #         for j in range(3):
        #             fin_data.append(i[j])

        #     # Right hand is detected
        #     # print("right detected")

        #     cv2.imwrite(file_path, resized)
        # else:
        #     for i in range(63):
        #         fin_data.append(0)
        #     # print("right not detected")






        # # fin_data
        # print("Len of fin data: should be 126: ", len(fin_data))
        # fin_label.append(folder.split("/")[-1])
        # print("len of final label: ", len(fin_label), "   its: ", fin_label[0])
        # try:
        #     # file_path = "data1/" + str(c) +  "left.png"
        #     file_path = "data1/left" + images
        #     resized = cv2.resize(cropped_images[0], (resize_dim, resize_dim))
        #     temp_data.append(left_area)
        #     cv2.imwrite(file_path, resized)
        # except:
        #     temp_data.append("0")
        #     continue
        # try:
        #     # file_path = "data1/" + str(c) + "right.png"
        #     file_path = "data1/right" + images
        #     resized = cv2.resize(cropped_images[1], (resize_dim, resize_dim))
        #     temp_data.append(right_area)
        #     cv2.imwrite(file_path, resized)
        # except:
        #     temp_data.append("0")
        #     continue








        print("finished writing")
        # results[0].show()
    # else:
        
        break
    if folder.split("/")[-1] == str(3):
        break
    # else:
    #     continue


Folder:  /home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/19
Image

image 1/1 /home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/19/165.jpg: 192x224 1 Right_Hand, 8.4ms
Speed: 0.5ms preprocess, 8.4ms inference, 1.0ms postprocess per image at shape (1, 3, 192, 224)
Found left and right bb



Going to find keypoints: 






Taking element of cropped images
Len of the taken cropped image:  0
Image

image 1/1 /home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/19/198.jpg: 192x224 (no detections), 9.4ms
Speed: 1.4ms preprocess, 9.4ms inference, 0.5ms postprocess per image at shape (1, 3, 192, 224)
Found left and right bb



Going to find keypoints: 






Taking element of cropped images
Len of the taken cropped image:  0
Image

image 1/1 /home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/19/246.jpg: 192x224 1 Right_Hand, 6.7ms
Speed: 1.6ms preprocess, 6.7ms inference, 2.2ms postprocess per 

I0000 00:00:1741325498.991405    8541 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741325499.010026    9419 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741325499.034975    8541 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741325499.054290    9436 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741325499.085275    8541 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741325499.106591    9453 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741325499.145112    8541 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741325499.164691    9470 gl_context.cc:344] GL version: 3.2 (O




Going to find keypoints: 






Taking element of cropped images
Len of the taken cropped image:  0
Image

image 1/1 /home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/19/214.jpg: 192x224 1 Left_Hand, 14.1ms
Speed: 1.2ms preprocess, 14.1ms inference, 1.3ms postprocess per image at shape (1, 3, 192, 224)
Found left and right bb



Going to find keypoints: 






Taking element of cropped images
Len of the taken cropped image:  269
Image

image 1/1 /home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/19/100.jpg: 192x224 1 Left_Hand, 16.1ms
Speed: 1.3ms preprocess, 16.1ms inference, 1.4ms postprocess per image at shape (1, 3, 192, 224)
Found left and right bb



Going to find keypoints: 






Taking element of cropped images
Len of the taken cropped image:  238
Image

image 1/1 /home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/19/37.jpg: 192x224 (no detections), 8.9ms
Speed: 1.1ms preprocess, 8.9ms inference, 0.

I0000 00:00:1741325499.211300    9487 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741325499.250548    8541 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741325499.269228    9504 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741325499.305228    8541 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741325499.325518    9521 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741325499.353267    8541 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741325499.368278    9538 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:174132549

image 1/1 /home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/19/123.jpg: 192x224 1 Right_Hand, 14.1ms
Speed: 0.8ms preprocess, 14.1ms inference, 2.5ms postprocess per image at shape (1, 3, 192, 224)
Found left and right bb



Going to find keypoints: 






Taking element of cropped images
Len of the taken cropped image:  0
Image

image 1/1 /home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/19/166.jpg: 192x224 1 Right_Hand, 13.6ms
Speed: 2.5ms preprocess, 13.6ms inference, 1.9ms postprocess per image at shape (1, 3, 192, 224)
Found left and right bb



Going to find keypoints: 






Taking element of cropped images
Len of the taken cropped image:  0
Image

image 1/1 /home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/19/244.jpg: 192x224 1 Right_Hand, 8.9ms
Speed: 2.5ms preprocess, 8.9ms inference, 0.9ms postprocess per image at shape (1, 3, 192, 224)
Found left and right bb



Going to find keypoints: 






Tak

...

I0000 00:00:1741325500.177152    8541 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741325500.210860    9793 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2
I0000 00:00:1741325500.273776    8541 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741325500.308056    9810 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2


image 1/1 /home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/19/101.jpg: 192x224 (no detections), 131.3ms
Speed: 34.0ms preprocess, 131.3ms inference, 23.3ms postprocess per image at shape (1, 3, 192, 224)
Found left and right bb



Going to find keypoints: 






Taking element of cropped images
Len of the taken cropped image:  0
Image



I0000 00:00:1741325500.587209    8541 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741325500.625718    9827 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2


image 1/1 /home/abhinav/Desktop/AIProjects/General Projects/SLT/sign_recognition/data/19/61.jpg: 192x224 (no detections), 11866.4ms
Speed: 78.2ms preprocess, 11866.4ms inference, 213.3ms postprocess per image at shape (1, 3, 192, 224)
Found left and right bb



Going to find keypoints: 






Taking element of cropped images
Len of the taken cropped image:  0
Image



I0000 00:00:1741325512.954504    8541 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1741325513.006057    9869 gl_context.cc:344] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3050 Laptop GPU/PCIe/SSE2


In [5]:
data

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [6]:
labels

[['19']]

In [7]:
print(len(data))

1


In [8]:
print(len(label_to_export))

NameError: name 'label_to_export' is not defined